## Creating Scripts

Now, let's try to create our first "script", which we can execute from the shell directly.

* Go the the shell
* Type `nano getTemp`: This will use the text editor `nano` and create a file called `getTemp`
* Type the command from the earlier module
```
#!/bin/sh
curl -s 'http://freegeoip.net/json/' | \
jq '"http://api.openweathermap.org/data/2.5/weather?q=" + .city + "," + .region_name + "&mode=json&units=imperial&appid=ffb7b9808e07c9135bdcc7d1e867253d"' | \
xargs curl -s | jq '.main.temp'
```
* Notice the addition of the first line `#!/bin/sh`. We will get back to that later.
* Type Ctrl+X to save the file and exit
* In the shell, type `chmod 700 getTemp`. This makes our file _executable_.
* Finally type `./getTemp` and see what happens.

#### Exercise

* Modify the script, to add an extra command that also prints the date
* Modify the script, so that the output goes into the file /home/ubuntu/temperature.txt


## Running Jobs in the Background

Sometimes, we would like to start a task, and let it run in the background. To do so, we simply add the character `&` at the end of the command. For example, if we want to run a long running task using grep, and store the results in a file, we can type:

`cd /home/ubuntu/data/`

`grep -R 'MORIMOTO' . > morimoto.txt &`

#### Standard output, Standard Error

When we run tasks in the background, it is often useful to separate the storing of the program output from the program errors. This is done by using the `2>` redirect operator, which redirects the error messages to the file of our choice.

`grep -R 'MORIMOTO' . > morimoto.txt 2> morimoto-errors.txt &`

If we prefer to store both standard output and standard error in the same file, we use the `2>&1` command:

`grep -R 'MORIMOTO' . > morimoto.txt 2>&1 &`

#### Nohup

When we use the `&` operator, the task runs in the background, but stops running the moment we logout from our ssh session. To allow the task to continue running, even after we log out, we can use the `nohup` command, as follows:

`nohup grep -R 'MORIMOTO' . > morimoto.txt 2> morimoto-errors.txt &`

### Exercise

Start downloading a big data set (e.g., the restaurant data set) using CURL. Use the -s option to put it in silent mode, and use the nohup command and the & operator to let the process run in the background.

## Cron: Scheduling Tasks

      
Cron is used to execute desired tasks (in the background) at designated times. 

A crontab is a simple text file with a list of commands meant to be run at specified times and these jobs will run regardless of whether the user is actually logged into the system. 

To use cron for tasks meant to run only for your user profile, add entries to your own user's crontab file. Start the crontab editor from a terminal window:

`sudo crontab -e`

### The structure of the crontab file

This is how a cron job is laid out:

minute (0-59), hour (0-23, 0 = midnight), day (1-31), month (1-12), weekday (0-6, 0 = Sunday), command

and each line of the crontab file has the following format:

`minute hour day_of_month month day_of_week   command`

Each of the parts is separated by a space, with the final part (the command) having one or more spaces in it. 
For example, you can run a backup of all your user accounts at 5 a.m every week with:

`0 5 * * 1 tar -zcf /var/backups/home.tgz /home/`

#### More examples

`01 04 1 1 1 /usr/bin/somedirectory/somecommand`

The above example will run /usr/bin/somedirectory/somecommand at 4:01am on January 1st plus every Monday in January. An asterisk (\*) can be used so that every instance (every hour, every weekday, every month, etc.) of a time period is used. Code:


`01 04 * * * /usr/bin/somedirectory/somecommand`

The above example will run /usr/bin/somedirectory/somecommand at 4:01am on every day of every month.

Comma-separated values can be used to run more than one instance of a particular command within a time period. Dash-separated values can be used to run a command continuously. For example:

`01,31 04,05 1-15 1,6 * /usr/bin/somedirectory/somecommand`

The above example will run /usr/bin/somedirectory/somecommand at 01 and 31 past the hours of 4:00am and 5:00am on the 1st through the 15th of every January and June.

The `/usr/bin/somedirectory/somecommand` text in the above examples indicates the task which will be run at the specified times. It is recommended that you use the full path to the desired commands as shown in the above examples. Enter which somecommand in the terminal to find the full path to somecommand. The crontab will begin running as soon as it is properly edited and saved.

You may want to run a script some number of times per time unit. For example if you want to run it every 10 minutes use the following crontab entry (runs on minutes divisible by 10: 0, 10, 20, 30, etc.)

`*/10 * * * * /usr/bin/somedirectory/somecommand`

which is also equivalent to the more cumbersome

`0,10,20,30,40,50 * * * * /usr/bin/somedirectory/somecommand`


(See https://help.ubuntu.com/community/CronHowto for more details)


### Exercise

* Use a cron job to keep track of the temperature in New York, running every minute. Use the OpenWeatherMap service and jq to get the temperature. Use the redirect operator to store the temperature in a text file called /home/ubuntu/data/nyc-temperatures.txt, appending a new line for every measurement.